In [ ]:
import pickle
import numpy as np
from collections import defaultdict
import random
import copy
import math
import itertools
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
import time
from collections import Counter

# 1. Load input

Load episode set $E$ with the users that retweeted each original tweet in the trace. 

Each episode $E_{s}$ includes the users that retweeted s, ordered chronologically, as they appear in the trace. The first user in each episode is the user that originally tweeted the tweet, and is denoted by $r_{s}$. Subsequent users in $E_{s}$ are users that retweeted s, either directly from user $r_{s}$ or from another user that appears in $E_{s}$  before them.

In [ ]:
lines = 500000
random.seed(10)

In [ ]:
E = pickle.load(open("./extracted/E"+ str(lines) + ".p", "rb"))

In [ ]:
for tweet in E:
    E[tweet] = list(dict.fromkeys(E[tweet]))

Load episode set $D$ with the users that retweeted each original tweet in the trace. 

In [ ]:
D = pickle.load(open("./extracted/D"+ str(lines) + ".p", "rb"))

Load the set of original tweets denoted by $S$. 

The set of original tweets is denoted by $S$, where |$S$| = S is the total number of original tweets

In [ ]:
S = pickle.load(open("./extracted/S"+ str(lines) + ".p", "rb"))

Load $U$ set xith unique users

In [ ]:
U = pickle.load(open("./extracted/U"+ str(lines) + ".p", "rb"))
U = list(U)

Load $M_{ij}$ variables that count number of episodes where the ordered pair (i,j) appears

In [ ]:
M = pickle.load(open("./extracted/M"+ str(lines) + ".p", "rb"))

# 2. Find important quantities

- Number of unique users N
- Number of episodes / original tweets S
- Number of active pairs (i,j)

For every active pair we have to count the $\sigma_{ij}$'s

In [ ]:
N = len(U)
print('Number of unique users N =', N)

In [ ]:
print('Number of episodes/original tweets S =',len(E))

In [ ]:
active_pairs_n = 0 
for i in M:
    active_pairs_n+=len(M[i])

In [ ]:
print('Number of active user pairs in the trace:', active_pairs_n, 'out of the', N*(N-1), 'possible pairs')

# 3. Constrained-EM algorithm

Define important functions 

In [ ]:
def flatten(obj):
    if type(obj) == list:
        return [l for L in obj for l in L]
    if type(obj) == dict:
        return [l for i in obj for l in obj[i].values()]
    if type(obj) == defaultdict:
        return [l for i in obj for l in obj[i].values()]

In [ ]:
def update_Q(M, Q, s, a, b, r):
    for i in M:
        for j in M[i]:
            e = M[i][j] * s[i][j]
            numer = r * (a**e) * ((1-a)**(M[i][j]-e))
            denom = r * (a**e) * ((1-a)**(M[i][j]-e)) + (1-r) * (b**e) * ((1-b)**(M[i][j]-e))
            Q[i][j] = numer/denom   
    return Q

In [ ]:
def update_a(M, Q, s):
    a = ((sum([M[i][j] * Q[i][j] * s[i][j]  for i in M for j in M[i]]))
        /((sum([M[i][j] * Q[i][j] for i in M for j in M[i]])))) 
    return(a)

In [ ]:
def update_b(M, Q, s):
    b = ((sum([M[i][j] * (1-Q[i][j]) * s[i][j]  for i in M for j in M[i]]))
        /((sum([M[i][j] * (1-Q[i][j]) for i in M for j in M[i]]))))
    return(b)

In [ ]:
def update_r(Q, N, active_pairs_n):
    r = (sum([Q[i][j] for i in Q for j in Q[i]]))/(active_pairs_n)      
    return(r)

In [ ]:
def update_s(model, M, Q, x, a, b, s, active_pairs):
    """ 
    This function updates the s_{ij} parameters of the optimization problemm.

    Parameters
    ----------
    model : PulpModel
        Model initialized from Pulp.
    M : dict
        Dictionary with M_{ij} values tha show how many times an ordered pair (i,j) appears in the trace.
    Q : dict
        Dictionary with the Q_{ij} values.
    W : dict
        Dictionary with the coefficients of the problem.
    x : dict
        Dictionary with the decision variables of the model.
    a : float 
            The true-positive utilization rate.
    b : float
            The false-positive utilization rate.
    s : dict
        Dictionary with the updated s_{ij} parameters.
    active_pairs : list
        Active (i,j) pairs.
        
    Returns
    ----------
    s : dict
        Dictionary with the updated s_{ij} parameters.
    """
    
    W = defaultdict(dict)
    for pair in active_pairs:
        i = pair[0]
        j = pair[1]
        W[i][j] = M[i][j]*((Q[i][j]*math.log(a/(1-a)))+ (1-Q[i][j])*math.log(b/(1-b))) + random.uniform(0,0.001)

    s = pulp_solve(model, active_pairs, W, x, s)
    return(s, W)

In [ ]:
def pulp_create_reduct(E, M):
    """ 
    This function initializes the optimization problem with reduced constraints. Runs the first time

    Returns
    ----------
    model : LpProblem
        Pulp model of the problem.
    x : Dict
        Dictionary with decision variables.
    """
    
    # initialize the maximization problem
    model = LpProblem(name="constr-newman", sense=LpMaximize)
    
    # for each episode in E, each line in constraints_list includes:
    # - for each user j in E, the users before them in index 0 and 
    # - the user itself in index 1
    
    sij = defaultdict(dict)
    constraints_list = []
    
    for s in D:
        for users_list in list(D[s].values()):
            index_now = list(D[s].values()).index(users_list)
            if index_now == 0:
                for u in list(D[s].values())[1]:
                    sij[D[s][0][0]][u] = 1
            else:
                for u in users_list:
                    u_before_l = list(D[s].values())[:index_now]
                    u_before = [item for sublist in u_before_l for item in sublist]
                    if u in u_before: u_before.remove(u)
                    constraints_list.append([u_before, u])

    len_b = len(constraints_list)

    # phase I: delete all constraints that include a pair with sij = 1
    
    for c in list(constraints_list):
        j = c[1]
        for i in c[0]:
            if i in sij and j in sij[i] and sij[i][j]==1:
                constraints_list.remove(c)
                break
                
    # for each episode in $E$, the constraints_dictionary includes:
    # - the user j as keys
    # - the users that come before them for each constraint that they appear in pos j as values
    
    constraints_list.sort()
    constraints_list = list(constraints_list for constraints_list,_ in itertools.groupby(constraints_list))
    constraints_dict = dict()
    ind = 0 
    for c in constraints_list:
            j = c[1]
            if j in constraints_dict:
                constraints_dict[j].append([c[0],ind])
            else:
                constraints_dict[j] = []
                constraints_dict[j].append([c[0],ind])
            ind+=1

    # phase II  ----------------------------
    
    for j in constraints_dict:
        for constraint1 in constraints_dict[j]:
            for constraint2 in constraints_dict[j]:
                if constraint1[1]!=constraint2[1]:
                    if set(constraint1[0]).issubset(constraint2[0]):
                        if constraint1[0]!=constraint2[0] and collections.Counter(constraint1[0]) != collections.Counter(constraint2[0]): 
                            if [constraint2[0],j] in constraints_list: # we may have already deleted it
                                constraints_list.remove([constraint2[0],j])
                    
    len_a = len(constraints_list)          
    
    # add decision variables and create active pairs and sij's randomly 
    
    active_pairs = []
    for i in M:
        for j in M[i]:
            if i not in sij or j not in sij[i]:
                active_pairs.append((i,j))
                sij[i][j] = random.uniform(0, 1)

    active_pairs = list(set(active_pairs))
        
    x = dict()
    x = {f"{pair[0]}-{pair[1]}":LpVariable(name=f"x{pair[0]}-{pair[1]}", lowBound=0, upBound=0.9999999999999) for pair in active_pairs}
    
    # add constraints in model
    for c in constraints_list: 
        j = c[1]
        tmp = 0
        for i in c[0]:
            tmp+=x[f"{i}-{j}"]
        model += (tmp >= 1)
    
    return model, x, active_pairs, sij

In [ ]:
def pulp_solve(model, active_pairs, W, x, s):
    """ 
    This function solves the optimization problem with PULP.
    
    Parameters
    ----------
    model : PulpModel
        Model initialized from pulp
    active_pairs : list
        Active pairs of the problem
    W : dict
        Dictionary with the coefficients of the problem 
    x : dict
        Dictionary with decision variables

    Returns
    ----------
    s : dict
        Dictionary with found s_{ij} parameters
    
    """
    
    maxv = max((set(max(list(i.values()) for i in W.values()))))
    c = maxv + 0.0001
    
    # update the objective function
    model += lpSum((W[i][j]-c)*x[f"{i}-{j}"] for i in W for j in W[i])
    # and solve it 
    status = model.solve()

    for var in x.values():
        i = var.name.split("_")[0]
        i = int(i.split("x")[1])
        j = int(var.name.split("_")[1])
        s[i][j] = var.value()
    
    return s
                    

In [ ]:
def constrainedem(eps, N, M, active_pairs_n):
        """ 
        This function is the main algorithm for path inference with constraints. 

        Parameters
        ----------
        eps : float
            Convergence criterion.
        N : int
            Number of users in the trace.
        M : dict
            Dictionary with M_{ij} values tha show how many times an ordered pair (i,j) appears in the trace.

        Returns
        ----------
        a : float 
            The true-positive utilization rate.
        b : float
            The false-positive utilization rate.
        r : float
            The uniform prior probability of the existence of an edge in any position between any pair of nodes.
        s : dict
            Dictionary with s_{ij} values tha show the limiting frequency that 
            user j reposts directly a post from i when the number of episodes goes to infinity.    
        Q : dict
            Dictionary with the Q_{ij} values that show the posterior probability of an edge existing between $i$ and $j$.
        """

        iterat = 1

        # initialize Q dictionary 
        Q = defaultdict(dict)

        # initialize a, b, r parameters randomly
        a = random.uniform(0.5, 1)
        b = random.uniform(0, 0.5)
        r = random.uniform(0, 1)
        
        # create pulp model with reduced constraints and variables...')
        model, x, active_pairs, s = pulp_create_reduct(E, M)
        
        # ======================== START algorithm ========================

        while True: # repeat until convergence
            # Step 1 ==== UPDATE VALUES ====

            # 1.1 -- Update Q dictionary
            Q = update_Q(M, Q, s, a, b, r)
            
            # 1.2 -- Update a,b,r,s_{ij} parameters according to Equations
            a = update_a(M, Q, s)
            b = update_b(M, Q, s)
            r = update_r(Q, N, active_pairs_n)

            if a ==1: 
                a = 0.9999999
                flag = True
            if b ==0: 
                b = 0.0001
                flag = True
            
            s, W  = update_s(model, M, Q, x, a, b, s, active_pairs)

            # Step 2 ==== CHECK CONVERGENCE ====

            if iterat > 1:
                new_q = np.array(flatten(Q))
                new_a = a
                new_b = b
                cost = sum(s[i][j]*M[i][j]*(Q[i][j]*math.log(a/(1-a))+(1-Q[i][j])*math.log(b/(1-b))) for i in s for j in s[i])
                new_cost = cost
                changeq = np.linalg.norm(new_q - old_q)  
                changea = abs(new_a-old_a)
                changeb = abs(new_b-old_b)
                changecost = abs(new_cost - old_cost)
                if changeq<eps:
                    Q_old = copy.deepcopy(Q)
                    old_q = np.array(flatten(Q_old))
                    old_a = a
                    old_b = b
                    old_cost = cost        
                    break
                else: 
                    Q_old = copy.deepcopy(Q)
                    old_q = np.array(flatten(Q_old))
                    old_a = a
                    old_b = b
                    old_cost = cost
                    iterat += 1
            if iterat == 1:
                flag = False
                old_q = np.array(flatten(Q))
                old_a = a
                old_b = b
                cost = sum(s[i][j]*M[i][j]*(Q[i][j]*math.log(a/(1-a))+(1-Q[i][j])*math.log(b/(1-b))) for i in s for j in s[i])
                old_cost = cost
                iterat+=1
                
        return a, b, r, s, Q

In [ ]:
eps = 0.001
a, b, r, s, Q = constrainedem(eps, N, M, active_pairs_n)

In [ ]:
try: 
    pickle.dump(Q, open("./extracted/Q_constrained" + str(lines) + ".p", "wb"))
except: 
    print("Unable to write to file")

In [ ]:
try: 
    pickle.dump(s, open("./extracted/s_constrained" + str(lines) + ".p", "wb"))
except: 
    print("Unable to write to file")